# 新增全市患者数量作为输入变量
借助各个street的患者数量求和实现

In [8]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [9]:
path = 'C:/Users/new/Desktop/Prediction-of-Containment-Events/数据/Baidu_Patient_Geocoding.csv'
data = pd.read_csv(path)

In [10]:
data.drop(['来源1','来源2'], axis=1, inplace=True)

In [11]:
data.head()
data.info()

,患者序号,性别,年龄,职业,家庭住址,分型,formatted_address,district,street,street_number,business,place_around,road1,road2,road3,经度,纬度
0,1,女,29,托辅机构工作人员,南开区华苑新城,轻型,天津市南开区华苑路59号,南开区,华苑路,59号,"华苑街,王顶堤,水上",0.0,宾水西道延长线,港宁西路,港宁东路,117.150067,39.081834
1,2,女,10,咸水沽第七小学学生,津南区咸水沽镇,轻型,天津市津南区丰收路,津南区,丰收路,NaN,咸水沽,1.0,丰收路,S317(二八公路),S317(南环路),117.401664,38.984801
2,3,男,10,咸水沽第七小学学生,津南区北闸口镇,轻型,天津市津南区北营路,津南区,北营路,NaN,NaN,2.0,北营路,广惠道,S106辅路,117.412547,38.956325
3,4,男,12,辛庄中学学生,津南区辛庄镇,轻型,天津市津南区津沽公路辅路,津南区,津沽公路辅路,NaN,辛庄,3.0,津沽公路辅路,S105(津沽公路),津沽公路,117.356617,39.032633
4,5,男,13,辛庄中学学生,津南区辛庄镇,轻型,天津市津南区津沽公路辅路,津南区,津沽公路辅路,NaN,辛庄,4.0,津沽公路辅路,S105(津沽公路),津沽公路,117.356617,39.032633


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 430 entries, 0 to 429
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   患者序号               430 non-null    int64  
 1   性别                 430 non-null    object 
 2   年龄                 430 non-null    object 
 3   职业                 91 non-null     object 
 4   家庭住址               427 non-null    object 
 5   分型                 352 non-null    object 
 6   formatted_address  427 non-null    object 
 7   district           427 non-null    object 
 8   street             427 non-null    object 
 9   street_number      70 non-null     object 
 10  business           416 non-null    object 
 11  place_around       427 non-null    float64
 12  road1              427 non-null    object 
 13  road2              425 non-null    object 
 14  road3              423 non-null    object 
 15  经度                 427 non-null    float64
 16  纬度                 427 non

In [12]:
path = 'C:/Users/new/Desktop/Prediction-of-Containment-Events/数据/train_street.csv'
train = pd.read_csv(path)

In [13]:
train.head()
train.info()

,日期,district,street,经停病例数,患者数量
0,2021/12/23 0:00,宝坻区,小侯庄桥,0,0
1,2021/12/24 0:00,宝坻区,小侯庄桥,0,0
2,2021/12/25 0:00,宝坻区,小侯庄桥,0,0
3,2021/12/26 0:00,宝坻区,小侯庄桥,0,0
4,2021/12/27 0:00,宝坻区,小侯庄桥,1,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34317 entries, 0 to 34316
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   日期        34317 non-null  object
 1   district  34317 non-null  object
 2   street    34317 non-null  object
 3   经停病例数     34317 non-null  int64 
 4   患者数量      34317 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 1.3+ MB


In [14]:
for index,row in train.iterrows():
    train.loc[index, '日期'] = datetime.strptime(row['日期'], "%Y/%m/%d %H:%M")

In [15]:
# 生成时间序列
start_timestamp = pd.Timestamp(2021,12,23)
end_timestamp = pd.Timestamp(2022,2,1)
time_series = pd.date_range(start_timestamp,end_timestamp)

In [16]:
time_series

DatetimeIndex(['2021-12-23', '2021-12-24', '2021-12-25', '2021-12-26',
               '2021-12-27', '2021-12-28', '2021-12-29', '2021-12-30',
               '2021-12-31', '2022-01-01', '2022-01-02', '2022-01-03',
               '2022-01-04', '2022-01-05', '2022-01-06', '2022-01-07',
               '2022-01-08', '2022-01-09', '2022-01-10', '2022-01-11',
               '2022-01-12', '2022-01-13', '2022-01-14', '2022-01-15',
               '2022-01-16', '2022-01-17', '2022-01-18', '2022-01-19',
               '2022-01-20', '2022-01-21', '2022-01-22', '2022-01-23',
               '2022-01-24', '2022-01-25', '2022-01-26', '2022-01-27',
               '2022-01-28', '2022-01-29', '2022-01-30', '2022-01-31',
               '2022-02-01'],
              dtype='datetime64[ns]', freq='D')

In [17]:
time_series[0] == train.loc[0, "日期"]

True

In [18]:
train.loc[0, "日期"]

datetime.datetime(2021, 12, 23, 0, 0)

In [19]:
time_series[0]

Timestamp('2021-12-23 00:00:00', freq='D')

In [20]:
total_num = []

In [21]:
for datetime in time_series:
    rows_index = train[(train['日期'] == datetime)].index.tolist()
    count = 0
    for ind in rows_index:
        count += train.loc[ind, '患者数量']
    total_num.append(count)

In [22]:
total_num

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 162,
 378,
 604,
 844,
 868,
 1096,
 1246,
 1436,
 1488,
 1388,
 1190,
 1322,
 1094,
 944,
 592,
 324,
 198,
 156,
 30,
 30,
 66,
 120,
 216,
 216,
 309]

In [16]:
train['全市总患者数量'] = 0

In [17]:
i = 0
length = len(total_num)
for index,row in train.iterrows():
    train.loc[index, '全市总患者数量'] = total_num[i]
    i += 1
    i = i % length

In [18]:
train

,日期,镇区,经停病例数,患者数量,经度,纬度,全市总患者数量
0,2021-12-23 00:00:00,东马圈镇,0,0,116.844191,39.444862,0
1,2021-12-24 00:00:00,东马圈镇,0,0,116.844191,39.444862,0
2,2021-12-25 00:00:00,东马圈镇,0,0,116.844191,39.444862,0
3,2021-12-26 00:00:00,东马圈镇,0,0,116.844191,39.444862,0
4,2021-12-27 00:00:00,东马圈镇,0,0,116.844191,39.444862,0
...,...,...,...,...,...,...,...
6227,2022-01-28 00:00:00,北仓镇,0,0,117.117081,39.233295,169
6228,2022-01-29 00:00:00,北仓镇,0,0,117.117081,39.233295,119
6229,2022-01-30 00:00:00,北仓镇,0,0,117.117081,39.233295,89
6230,2022-01-31 00:00:00,北仓镇,0,0,117.117081,39.233295,68


In [19]:
output_path='C:/Users/new/Desktop/train_addvar.csv'
train.to_csv(output_path,sep=',',index=False,header=False,encoding='utf_8_sig')